<a href="https://colab.research.google.com/github/luminyanko/DSL/blob/main/job3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Job #3
Savenko Valeriia

Let's say that context free grammar is such this:

```
{'toks': set(token), 'vars': dict(var: definition), 'hvar': var}
token : (class, value)
class : int
value : str
var : str                 # non-terminal name
definition : list(rule)
rule : list(var | token)  # right part of the rule
```



Функція FIRST повертає набір терміналів, які можна вирахувати з нетерміналу, який ми їй передали. Використовуючи той факт, що ми вилучили ліву рекурсію з нашої граматики, ми можемо легко реалізувати цю функцію повторно, оскільки є докази того, що ми не застрягнемо в нескінченному циклі. Отже, якщо ми отримали нетермінал A, ми починаємо перебирати правила, в яких ліва частина має тільки A, і якщо перший символ правила є терміналом h - ми просто додаємо його до набору.

In [18]:
def FIRST(grammar, symbol):
  tokens = grammar['toks']
  variables = grammar['vars']
  result = set()
  for rule in variables[symbol]:
    if rule[0] in tokens:
      result.add(rule[0])
    else:
      result |= FIRST(grammar, rule[0])
  return result

#FOLLOW 
FOLLOW(X) — це набір терміналів, які можуть з’являтися відразу після X у певному виведенні. Щоб обчислити FOLLOW(X), треба знайти усі вихідні термінали, включаючи X, у правій частині, і отримати наступні символи в цих вихідніх терміналах. Якщо після X немає символів у деякому виведенні A -> pX, то FOLLOW(X) те саме, що FOLLOW(A)


In [21]:
def FOLLOW(grammar, symbol):
  tokens = grammar['toks']
  variables = grammar['vars']
  useful_vars = []
  for nt, val in variables.items():
    for rule in val:
      if symbol in rule:
        useful_vars.append((nt, rule))
  
  result = set()
  for p in useful_vars:
    lhs, rhs = p
    idx = rhs.index(symbol)
    if idx + 1 < len(rhs):
      next_symbol = rhs[idx + 1]
      if next_symbol in tokens:
        result.add(next_symbol)
      else:
        result |= FIRST(grammar, next_symbol)
    else:
      if lhs != symbol:
        result |= FOLLOW(grammar, lhs)
  return result

Example

In [31]:
grammar = {
    'toks': {(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd'), (4, 'f'), (5, 'g'), (6, 'h'), (7, 'e')},
    'vars': {
        'A': [[(0, 'a') ,'B', 'C', (6, 'h')]],
        'B': [[(2, 'c'), 'C']], 
        'C': [[(1, 'b'), 'D']],
        'D': [['C', 'F']],
        'E': [[(5, 'g')], [(7, 'e')]],
        'F': [[(4, 'f')], [(6, 'h')]],
    },
    'hvar': 'A',
}

In [32]:
print(FIRST(grammar, 'C'))
print(FOLLOW(grammar, 'C'))


{(1, 'b')}
{(4, 'f'), (1, 'b'), (6, 'h')}
